# Вычислительный пракртикум

## Задание №1
## Схема Гаусса решения СЛАУ. Нахождение обратной матрицы. Вычисление числа обусловленности

### Ковальчуков Александр
### 321 группа

### Вариант №6

In [161]:
import numpy as np

# Параметры задачи

In [162]:
__A = np.array([[ 9.016024, 1.082197, -2.783575],
              [ 1.082197, 6.846595,  0.647647],
              [-2.783575, 0.647647,  5.432541]])

__C = __A.copy()
__C[0, 0] *= 10**-8

__b = np.array([[-1.340873],[4.179164],[5.478007]])

n, m  = __A.shape

epsilon = 1e-10

In [163]:
def check(A, b, x):
    print("Матрица:\n", A)
    print("Столбец свободных членов:\n", b)

    print("Решение системы:")
    print("x =\n", x)

    dx = np.dot(A, x) - b

    print("Вектор невязки:")
    print("dx =\n", dx)


# Схема Гаусса единственного деления

In [164]:
def solve_Gauss(A, b):
    A = np.hstack((A, b))
    x = np.zeros((3, 1))

    # Прямой ход
    for k in range(n):
        if abs(A[k, k]) < epsilon:
            raise ZeroDivisionError
        A[k, :] /= A[k, k]
        for i in range(k + 1, n):
            A[i, :] -= A[k, :] * A[i, k]

    # Обратный ход
    for i in range(0, n)[::-1]:
        x[i] = A[i, n] - np.dot(A[i, :-1], x)

    return x

# Схема Гаусса с выбором главного элемента по столбцу и строке.

In [165]:
def transform(A, b):

    # делаем из квадратной матрицы и столбца расширенную матрицу
    A = np.hstack((A, b))

    # добавляем служебную строчку к матрице
    c = np.array(range(A.shape[1])).reshape((1, A.shape[1]))
    A = np.vstack((A, c))

    # переставляем строки и столбцы
    for k in range(n):
        for p in range(k, n):
            for q in range(k, m):
                if abs(A[p, q]) > abs(A[k, k]):
                    A[[k, p], :] = A[[p, k], :]
                    A[:, [k, q]] =  A[:, [q, k]]

    # разбираем матрицу обратно на запчасти
    b = A[:-1, -1].reshape((n, 1))
    c = A[-1, :-1]
    A = A[:-1, :-1]

    # отладочный вывод
    # print("A =\n", A)
    # print("b =\n", b)
    # print("c =\n", c)
    return A, b, c

In [166]:
def transform_solver(A, b):
    A, b, c = transform(A, b)

    x = np.zeros((n, 1))
    # Решение по схеме Гаусса подготовленной матрицы
    unordered_x = solve_Gauss(A.copy(), b.copy())

    # Обратная перестановка элементов
    for i in range(n):
        x[int(c[i])] = unordered_x[i]
    x = np.array(x)
    return x


# LU-разложение

In [167]:
def LU(A):
    L = np.zeros(A.shape)
    U = np.zeros(A.shape)

    # LU-разложение
    for i in range(n):
        for j in range(i, n):
            L[j, i] = A[j, i] - np.dot(L[j, :i], U[:i, i])
            U[i, j] = (A[i, j] - np.dot(L[i, :i], U[:i, j])) / L[i, i]

    print("L=\n", L)
    print("U=\n", U)
    print("Проверка правильности разложения:")
    print("LU - A =\n", np.dot(L, U) - __A)

    return L, U

# Нахождение обратной матрицы при помощи LU-разложения

In [168]:
def inv(A):
    A_inv = np.zeros(A.shape)
    for i in range(n):
        b = np.zeros((n, 1))
        b[i] = 1
        y = solve_Gauss(L, b)
        x = solve_Gauss(U, y)
        A_inv[:, [i]] = x
    return A_inv


# Вычисление числа обусловленности

In [169]:
def norm1(A):
    n, m = A.shape
    norm = 0
    for j in range(n):
        s = 0
        for i in range(m):
            s += abs(A[i, j])
        norm = max(norm, s)
    return norm


def cond(A):
    return norm1(A) * norm1(inv(A))

# Тестрирование программы

## Схема Гаусса

In [170]:
print("Решение системы с матрицей A:")
x = solve_Gauss(__A.copy(), __b.copy())
check(__A, __b, x)
print()
print("Решение системы с матрицей C:")
x = solve_Gauss(__C.copy(), __b.copy())
check(__C, __b, x)

Решение системы с матрицей A:
Матрица:
 [[ 9.016024  1.082197 -2.783575]
 [ 1.082197  6.846595  0.647647]
 [-2.783575  0.647647  5.432541]]
Столбец свободных членов:
 [[-1.340873]
 [ 4.179164]
 [ 5.478007]]
Решение системы:
x =
 [[0.10000016]
 [0.49999994]
 [1.00000009]]
Вектор невязки:
dx =
 [[-2.22044605e-16]
 [-8.88178420e-16]
 [ 0.00000000e+00]]

Решение системы с матрицей C:
Матрица:
 [[ 9.016024e-08  1.082197e+00 -2.783575e+00]
 [ 1.082197e+00  6.846595e+00  6.476470e-01]
 [-2.783575e+00  6.476470e-01  5.432541e+00]]
Столбец свободных членов:
 [[-1.340873]
 [ 4.179164]
 [ 5.478007]]
Решение системы:
x =
 [[-0.42368555]
 [ 0.60939167]
 [ 0.71862795]]
Вектор невязки:
dx =
 [[-2.22044605e-16]
 [ 4.03274747e-10]
 [ 3.19230864e-09]]


## Схема Гаусса c выбором главного элемента по строке и столбцу

In [171]:
print("Решение системы с матрицей A:")
x = transform_solver(__A.copy(), __b.copy())
check(__A, __b, x)
print()
print("Решение системы с матрицей C:")
x = transform_solver(__C.copy(), __b.copy())
check(__C, __b, x)

Решение системы с матрицей A:
Матрица:
 [[ 9.016024  1.082197 -2.783575]
 [ 1.082197  6.846595  0.647647]
 [-2.783575  0.647647  5.432541]]
Столбец свободных членов:
 [[-1.340873]
 [ 4.179164]
 [ 5.478007]]
Решение системы:
x =
 [[0.10000016]
 [0.49999994]
 [1.00000009]]
Вектор невязки:
dx =
 [[-2.22044605e-16]
 [-8.88178420e-16]
 [ 0.00000000e+00]]

Решение системы с матрицей C:
Матрица:
 [[ 9.016024e-08  1.082197e+00 -2.783575e+00]
 [ 1.082197e+00  6.846595e+00  6.476470e-01]
 [-2.783575e+00  6.476470e-01  5.432541e+00]]
Столбец свободных членов:
 [[-1.340873]
 [ 4.179164]
 [ 5.478007]]
Решение системы:
x =
 [[-0.42368555]
 [ 0.60939167]
 [ 0.71862795]]
Вектор невязки:
dx =
 [[-2.22044605e-16]
 [ 0.00000000e+00]
 [ 8.88178420e-16]]


## LU-разложение
### Решение системы и обращение матрицы

In [172]:
print("Решение системы с матрицей A:")
L, U = LU(__A.copy())
y = solve_Gauss(L, __b.copy())
x = solve_Gauss(U, y)
check(__A, __b, x)
print()
print("Обратная матрица для A:")
print("inv(A)=\n", inv(__A.copy()))
print("Проверка:")
print("A*inv(A) =\n", np.dot(__A.copy(), inv(__A.copy())))
print("\n_____________________________\n")
print("Решение системы с матрицей C:")
L, U = LU(__C.copy())
y = solve_Gauss(L, __b.copy())
x = solve_Gauss(U, y)
check(__C, __b, x)
print()
print("Обратная матрица для C:")
print("inv(C)=\n", inv(__C.copy()))
print("Проверка:")
print("C*inv(C) =\n", np.dot(__C.copy(), inv(__C.copy())))


Решение системы с матрицей A:
L=
 [[ 9.016024    0.          0.        ]
 [ 1.082197    6.71669846  0.        ]
 [-2.783575    0.98176063  4.42964886]]
U=
 [[ 1.          0.1200304  -0.30873642]
 [ 0.          1.          0.14616714]
 [ 0.          0.          1.        ]]
Проверка правильности разложения:
LU - A =
 [[0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.11022302e-16 0.00000000e+00]]
Матрица:
 [[ 9.016024  1.082197 -2.783575]
 [ 1.082197  6.846595  0.647647]
 [-2.783575  0.647647  5.432541]]
Столбец свободных членов:
 [[-1.340873]
 [ 4.179164]
 [ 5.478007]]
Решение системы:
x =
 [[0.10000016]
 [0.49999994]
 [1.00000009]]
Вектор невязки:
dx =
 [[-2.22044605e-16]
 [-8.88178420e-16]
 [ 0.00000000e+00]]

Обратная матрица для A:
inv(A)=
 [[ 0.13709197 -0.02863689  0.07365842]
 [-0.02863689  0.15370581 -0.03299746]
 [ 0.07365842 -0.03299746  0.22575153]]
Проверка:
A*inv(A) =
 [[ 1.00000000e+00  3.45673394e-17 -1.3535

## Вычисление числа обусловленности

In [173]:
print("cond(A) =", cond(__A).copy())
print("cond(C) =", cond(__C).copy())

cond(A) = 13.065331839479951
cond(C) = 8.99005114981671


При решении СЛАУ, LU-разложние показывает слегка лучшую точность (в терминах нормы вектора невязки) по сравнению с прямым методом Гаусса.
В то же время, Схема с выбором главного элемента может в порядки раз уменьшить норму невязки в случае, когда ведущим элементом прямого метода Гаусса оказывается малое по модулю число.



